In [1]:
import config
import utils
import csv
# import random

from collections import defaultdict

import torch

from experiment import Learner

/home/km55359/.conda/envs/kmisra/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
implausibles = set(config.IMPLAUSIBLE['do'] + config.IMPLAUSIBLE['pp'])

In [5]:
# generalization = utils.read_json("../data/experiments/single_stimuli_dative_simulation/sentences/generalization.json")
generalization = utils.read_json("../data/experiments/single_stimuli_dative_simulation_valtest_vbd_no_markedness_no_discourse_bal_gen/sentences/adaptation.json")

In [6]:
# balanced_genset = []
# for key, datives in generalization.items():
#     if "do" in datives and "pp" in datives:
#         for do_sent, pp_sent in zip(datives["do"], datives["pp"]):
#             balanced_genset.append({"key": key, "do": do_sent, "pp": pp_sent})

# random.seed(42)

balanced_genset = []
for key, datives, in generalization.items():
    # pasdpasi
    # skip 3rd and 7th positions in the key string
    condensed_key = key[:3] + key[4:7] + key[8:]
    if condensed_key not in implausibles:
        # randomly sample 5 do and 5 pp sentences
        for do_sent, pp_sent in zip(datives["do"], datives["pp"]):
            balanced_genset.append({"key": key, "do": do_sent, "pp": pp_sent})
    

In [7]:
# balanced_genset
len(balanced_genset)
# balanced_genset

640

In [8]:
# with open("../data/balanced_generalization_set_three_features.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerow(["key", "dative", "sentence"])
#     for row in balanced_genset:
#         writer.writerow((row['key'], "do", row['do']))
#         writer.writerow((row['key'], "pp", row['pp']))

with open("../data/balanced_generalization_set_four_features.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["key", "dative", "sentence"])
    for row in balanced_genset:
        writer.writerow((row['key'], "do", row['do']))
        writer.writerow((row['key'], "pp", row['pp']))

In [2]:
target = torch.load("../data/results/single_stimuli_dative_simulation_valtest_vbd_discourse_theme_given_template_1/smolm-autoreg-bpe-seed_6/raw/1_1_1_0.1_do_results/verb_embedding.pt").to("cpu")



# /home/km55359/encouraging-exposures/data/results/single_stimuli_dative_simulation_valtest_vbd_discourse_theme_given_template_1/smolm-autoreg-bpe-seed_6/raw/1_1_1_0.1_do_results/verb_embedding.pt

In [7]:
lm = Learner("kanishka/smolm-autoreg-bpe-seed_6")
lm.add_tokens()
lm.freeze_full()

In [10]:
lm.token_score("she [verb] the ball to me .")

[[('<s>', 0.0),
  ('she', -4.904932975769043),
  ('[verb]', -13.991379737854004),
  ('the', -7.50151252746582),
  ('ball', -5.1817498207092285),
  ('to', -2.408505439758301),
  ('me', -1.0440807342529297),
  ('.', -0.6565561294555664)]]

In [6]:
lm.lm.model.resize_token_embeddings().weight[lm.new_index] = target

lm.token_score("she [verb] the ball to me .")

[[('<s>', 0.0),
  ('she', -4.904930114746094),
  ('[verb]', -6.0155487060546875),
  ('the', -6.001806259155273),
  ('ball', -7.697290420532227),
  ('to', -3.8050122261047363),
  ('me', -1.605154037475586),
  ('.', -0.5210390090942383)]]